<a href="https://colab.research.google.com/github/hugneiTP/Imersao-IA-Alura/blob/main/Projeto_Agente.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
%pip -q install google-genai

In [ ]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [ ]:
# Instalar Framework ADK de agentes do Google ################################################
%pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
########################################################
# --- Agente 1: Analisador de Notícias Financeiras --- #
########################################################
def agente_noticias_mercado(ativo, data_de_hoje):
    agente = Agent(
        name="agente_noticias_mercado",
        model="gemini-2.0-flash",
        instruction=f"""
        Você é um analista de mercado financeiro. Use a ferramenta de busca Google_search para coletar
        as notícias mais relevantes e recentes (últimos 15 dias) sobre o ativo {ativo}.
        Foque em eventos econômicos, políticos ou setoriais que possam afetar o preço do ativo.
        Classifique o viés das notícias como positivo, negativo ou neutro.
        Forneça um resumo breve e direto.
        """,
        description="Agente que coleta e resume notícias financeiras de ativos.",
        tools=[google_search],
    )

    entrada = f"Ativo: {ativo}\nData: {data_de_hoje}"
    return call_agent(agente, entrada)


In [16]:
########################################################
# --- Agente 2: Analista Técnico --- #
########################################################

def agente_analise_tecnica(ativo):
    agente = Agent(
        name="agente_analise_tecnica",
        model="gemini-2.0-flash",
        instruction=f"""
        Você é um analista técnico de investimentos. Sua tarefa é interpretar gráficos e padrões de preço
        (assuma que você tem acesso aos principais indicadores técnicos, como médias móveis, RSI, MACD, etc.).

        Faça uma análise técnica baseada no ativo {ativo}, usando dados recentes como referência.
        Com base nessa análise, identifique se há tendência de alta, baixa ou indefinição.
        Diga se há sinais de compra, venda ou manutenção.
        """,
        description="Agente especialista em análise técnica de ações e ativos."
    )

    entrada = f"Ativo: {ativo}"
    return call_agent(agente, entrada)


In [ ]:
########################################################
# --- Agente 3: Recomendador de Investimento --- #
########################################################
def agente_recomendador(ativo, noticias, tecnica):
    agente = Agent(
        name="agente_recomendador",
        model="gemini-2.0-flash",
        instruction=f"""
        Você é um consultor de investimentos que toma decisões com base em análise técnica e notícias de mercado.
        Seu objetivo é combinar essas informações para emitir uma recomendação clara sobre o ativo {ativo}.

        Possíveis respostas:
        - Comprar
        - Vender
        - Manter

        Além da recomendação, forneça uma justificativa de 2 a 3 parágrafos.
        Evite linguagem muito técnica.
        """,
        description="Agente que decide se o ativo deve ser comprado, vendido ou mantido."
    )

    entrada = f"Ativo: {ativo}\nNotícias: {noticias}\nAnálise Técnica: {tecnica}"
    return call_agent(agente, entrada)


In [17]:
########################################################
# --- Agente 4: Explicador para Leigos --- #
########################################################
def agente_explicador(ativo, recomendacao):
    agente = Agent(
        name="agente_explicador",
        model="gemini-2.0-flash",
        instruction=f"""
        Você é um educador financeiro e deve explicar para um investidor iniciante a seguinte recomendação de investimento:

        Explique de forma clara e simples o que significa a recomendação sobre o ativo {ativo}, por que ela foi feita,
        e que cuidados a pessoa deve ter (ex: não investir sem entender riscos).

        Evite jargões. Use metáforas ou analogias se necessário.
        """,
        description="Agente que traduz a recomendação para linguagem acessível ao público geral."
    )

    entrada = f"Ativo: {ativo}\nRecomendação detalhada: {recomendacao}"
    return call_agent(agente, entrada)


In [ ]:
########################################################
# --- Bloco final: Orquestrador --- #
########################################################
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("📈 Iniciando o Advisor de Investimentos com 4 Agentes 📊")

ativo = input("🔎 Digite o código do ativo ou nome da empresa que você quer analisar (ex: PETR4, AAPL, BTC): ")

if not ativo:
    print("❗Você esqueceu de digitar o ativo.")
else:
    print(f"\n🔍 Buscando informações sobre: {ativo}...\n")

    noticias = agente_noticias_mercado(ativo, data_de_hoje)
    print("\n--- Agente 1: Notícias de Mercado ---\n")
    display(to_markdown(noticias))

    tecnica = agente_analise_tecnica(ativo)
    print("\n--- Agente 2: Análise Técnica ---\n")
    display(to_markdown(tecnica))

    recomendacao = agente_recomendador(ativo, noticias, tecnica)
    print("\n--- Agente 3: Recomendação ---\n")
    display(to_markdown(recomendacao))

    explicacao = agente_explicador(ativo, recomendacao)
    print("\n--- Agente 4: Explicação para Leigos ---\n")
    display(to_markdown(explicacao))

    print("\n✅ Análise completa finalizada.")


📈 Iniciando o Advisor de Investimentos com 4 Agentes 📊
